In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# ================= 0. 環境安裝區 =================

print("⏳ 正在安裝系統依賴 (FFmpeg)...")
# 安裝 FFmpeg (音訊處理的核心工具，Pydub 和 Demucs 都需要它)
!sudo apt-get install -y ffmpeg

print("⏳ 正在安裝 Python 套件...")
# 1. Demucs: 用於音源分離 (Meta 開發)
# 2. stable-ts: 用於 Stable Whisper (精確時間軸字幕)
# 3. gradio_client: 用於調用遠端 RVC 模型 API
# 4. pydub: 用於音訊剪輯與合成
# 5. yt-dlp: (選用) 雖然你的代碼沒直接用，但處理媒體通常建議安裝
!pip install -q demucs stable-ts gradio_client pydub yt-dlp

print("⏳ 正在配置 Wav2Lip 環境...")
# 檢查是否已經 Clone 過 Wav2Lip，避免重複 Clone
import os

if not os.path.exists("/content/Wav2Lip"):
    !git clone https://github.com/Rudrabha/Wav2Lip.git

# 安裝 Wav2Lip 的依賴庫
!cd Wav2Lip && pip install -r requirements.txt

# ⚠️ 修正 Wav2Lip 常見的 librosa 版本衝突問題 (保持穩定性)
!pip install librosa==0.9.1 numba==0.56.4 numpy==1.23.5

# 下載 Wav2Lip 臉部偵測所需的權重檔 (這是最常被遺忘的一步！)
!wget -nc "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "/content/Wav2Lip/face_detection/detection/sfd/s3fd.pth"

# 掛載 Google Drive (如果還沒掛載的話)
from google.colab import drive

if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")

print("✅ 所有環境安裝完成！")

In [5]:
# Cell 2: 全流程

import os
import shutil
import subprocess
import torch
import stable_whisper
from pathlib import Path
from gradio_client import Client, handle_file
from pydub import AudioSegment

# ================= 1. 參數設定區 =================

# [Input 1] 原曲路徑 (取伴奏)
input_original_song = "/content/drive/MyDrive/demo/demo_original.mp3"

# [Input 2] 您的翻唱路徑 (取人聲)
input_user_cover = "/content/drive/MyDrive/demo/demo_original.mp3"

# RVC 模型資料
model_pth = "/content/drive/MyDrive/demo/model.pth"
model_index = "/content/drive/MyDrive/demo/model.index"
f0_change = 0

# 人物圖片
image_input_path = "/content/drive/MyDrive/demo/godtone.jpg"

# 🎵 原歌詞 (主要對齊對象)
lyrics_input = """
The snow glows white on the mountain tonight
Not a footprint to be seen
A kingdom of isolation
and it looks like I'm the queen
The wind is howling like this swirling storm inside
Couldn't keep it in
Heaven knows I've tried
"""

# 👂 空耳歌詞 (行數需與上方一致)
soramimi_input = """
特斯諾 哥羅斯 外特 噢恩 德 馬恩廷 托奈特
諾 诶 馮特普 入 唐 貝 宋
啊 金德姆 哦夫 依瑟拉申
安 依特 花格 萊克 爱姆 德 奎因
德 威恩 德 瑙林 萊克 息 斯沃林 斯托姆 内萨德
扣得 肯 爱特 因
海文 奈兹 爱弗 特赖德
"""

# 輸出位置
output_dir = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output"

# ==========================================================

%cd /content/Wav2Lip
os.makedirs(output_dir, exist_ok=True)
temp_dir = Path("/content/temp_dual_process")
if temp_dir.exists():
    shutil.rmtree(temp_dir)
temp_dir.mkdir(parents=True, exist_ok=True)

song_name = Path(input_user_cover).stem
print(f"🚀 終極流水線啟動！目標：{song_name}")

# --- 階段 1: 音源分離 (Demucs) ---
print("\n=== [1/5] 音源分離處理 ===")

# 1.1 原曲 -> 伴奏
print(f"   ✂️ 分離原曲伴奏: {os.path.basename(input_original_song)}...")
subprocess.run(
    [
        "demucs",
        "-n",
        "htdemucs",
        "--two-stems=vocals",
        "-o",
        str(temp_dir / "bgm_source"),
        input_original_song,
    ],
    check=True,
    stdout=subprocess.DEVNULL,
)
bgm_name = Path(input_original_song).stem
final_inst_path = temp_dir / "bgm_source" / "htdemucs" / bgm_name / "no_vocals.wav"
if not final_inst_path.exists():
    raise FileNotFoundError("原曲分離失敗")
print("     ✅ 已取得原曲伴奏")

# 1.2 翻唱 -> 人聲
print(f"   ✂️ 提取翻唱人聲: {os.path.basename(input_user_cover)}...")
subprocess.run(
    [
        "demucs",
        "-n",
        "htdemucs",
        "--two-stems=vocals",
        "-o",
        str(temp_dir / "vocal_source"),
        input_user_cover,
    ],
    check=True,
    stdout=subprocess.DEVNULL,
)
vocal_name = Path(input_user_cover).stem
user_vocals_path = temp_dir / "vocal_source" / "htdemucs" / vocal_name / "vocals.wav"
if not user_vocals_path.exists():
    raise FileNotFoundError("人聲提取失敗")
print("     ✅ 已取得乾淨人聲")


# --- 階段 2: AI 變聲 (RVC) ---
print("\n=== [2/5] AI 變聲轉換 (User -> Godtone) ===")
try:
    client = Client("r3gm/RVC_ZERO")
    result = client.predict(
        [handle_file(str(user_vocals_path))],
        handle_file(model_pth),
        "rmvpe+",
        f0_change,
        handle_file(model_index),
        0.75,
        3,
        0.25,
        0.5,
        False,
        False,
        "wav",
        1,
        api_name="/run",
    )
    converted_vocal_path = result[0] if isinstance(result, list) else result
    print("   ✅ 變聲成功！")
except Exception as e:
    print(f"❌ RVC 錯誤: {e}")
    raise e


# --- 階段 3: 混音 (Mix) ---
print("\n=== [3/5] 混音 (AI 人聲 + 原曲伴奏) ===")
voc_audio = AudioSegment.from_file(converted_vocal_path)
bgm_audio = AudioSegment.from_file(str(final_inst_path))
final_audio = voc_audio.overlay(bgm_audio)

ai_cover_path = os.path.join(output_dir, f"{song_name}_Godtone_Ver.wav")
final_audio.export(ai_cover_path, format="wav")
print(f"   💾 混音完成: {ai_cover_path}")


# --- 階段 4: Wav2Lip 視覺生成 ---
print("\n=== [4/5] 視覺生成 (Wav2Lip) ===")
lipsync_video_path = os.path.join(output_dir, f"{song_name}_lipsync.mp4")

if not os.path.exists(lipsync_video_path):
    print("   ⏳ 正在生成嘴型影片...")
    cmd = f'python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "{image_input_path}" --audio "{ai_cover_path}" --outfile "{lipsync_video_path}" --resize_factor 1'
    subprocess.run(cmd, shell=True, check=True)
else:
    print("   ⚠️ 嘴型影片已存在，跳過。")


# --- 階段 5: 雙軌字幕與最終合成 ---
print("\n=== [5/5] 雙軌字幕生成與合成 ===")
ass_path = os.path.join(output_dir, f"{song_name}.ass")
final_video_path = os.path.join(output_dir, f"{song_name}_Final_Ultimate.mp4")

# 5.1 字幕對齊 (使用雙軌邏輯)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = stable_whisper.load_model("medium", device=device)
clean_lyrics = lyrics_input.strip()
clean_soramimi = soramimi_input.strip()

if clean_lyrics and clean_soramimi:
    print("   🎯 正在對齊雙軌歌詞...")
    # 聽混音檔，對齊原歌詞
    result = model.align(
        ai_cover_path, text=clean_lyrics, language="en"
    )  # 英文歌設 en, 中文設 zh

    # 呼叫 Cell 1 定義的雙軌函數
    final_segments = realign_dual_tracks(result, clean_lyrics, clean_soramimi)
    generate_ktv_ass_dual(final_segments, ass_path)
else:
    print("❌ 錯誤：歌詞或空耳為空")

# 5.2 FFmpeg 合成
print("   🎬 渲染最終影片...")
shutil.copy(lipsync_video_path, "temp_src.mp4")
shutil.copy(ass_path, "temp_sub.ass")

ffmpeg_cmd = [
    "ffmpeg",
    "-y",
    "-v",
    "error",
    "-i",
    "temp_src.mp4",
    "-i",
    ai_cover_path,
    "-vf",
    "ass=temp_sub.ass",
    "-map",
    "0:v",
    "-map",
    "1:a",
    "-c:v",
    "libx264",
    "-preset",
    "medium",
    "-crf",
    "18",
    "-c:a",
    "aac",
    "-b:a",
    "192k",
    "-shortest",
    final_video_path,
]

try:
    subprocess.run(ffmpeg_cmd, check=True)
    print("\n" + "=" * 40)
    print(f"🎉🎉🎉 恭喜！終極版影片製作完成！")
    print(f"📂 檔案路徑: {final_video_path}")
    print("=" * 40)
except Exception as e:
    print(f"❌ 合成失敗: {e}")

if os.path.exists("temp_src.mp4"):
    os.remove("temp_src.mp4")
if os.path.exists("temp_sub.ass"):
    os.remove("temp_sub.ass")

/content/Wav2Lip
🚀 終極流水線啟動！目標：demo_original

=== [1/5] 音源分離處理 ===
   ✂️ 分離原曲伴奏: demo_original.mp3...
     ✅ 已取得原曲伴奏
   ✂️ 提取翻唱人聲: demo_original.mp3...
     ✅ 已取得乾淨人聲

=== [2/5] AI 變聲轉換 (User -> Godtone) ===
Loaded as API: https://r3gm-rvc-zero.hf.space ✔
   ✅ 變聲成功！

=== [3/5] 混音 (AI 人聲 + 原曲伴奏) ===
   💾 混音完成: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Godtone_Ver.wav

=== [4/5] 視覺生成 (Wav2Lip) ===
   ⚠️ 嘴型影片已存在，跳過。

=== [5/5] 雙軌字幕生成與合成 ===
   🎯 正在對齊雙軌歌詞...


/usr/local/lib/python3.12/dist-packages/stable_whisper/whisper_compatibility.py:273: UserWarning: The installed version of Whisper might be incompatible.
The detected version appears to be installed from the repository which can have compatibility issues due to multiple commits sharing the same version number. It is recommended to install version 20250625 from PyPI.
To prevent errors and performance issues, install the latest compatible version: `pip install openai-whisper==20250625` 
Use `ignore_compatibility=True` to ignore this warning.
  warnings.warn(compatibility_warning)
Align: 100%|██████████| 29.86/29.86 [00:00<00:00, 50.82sec/s]
Adjustment: 100%|██████████| 27.4/27.4 [00:00<00:00, 19337.70sec/s]


   🧩 雙軌字幕重組中...
   ✨ ASS 字幕檔已生成: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original.ass
   🎬 渲染最終影片...

🎉🎉🎉 恭喜！終極版影片製作完成！
📂 檔案路徑: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Final_Ultimate.mp4


In [6]:
# ================= 🚑 字幕消失修復與除錯工具 =================
import os
import subprocess

# 1. 安裝中文字體 (解決嫌疑一)
print("📦 正在安裝中文字體 (Noto Sans CJK)...")
!sudo apt-get update -qq
!sudo apt-get install -y fonts-noto-cjk
!fc-cache -fv
print("✅ 字體安裝完成！")

# 2. 檢查 ASS 檔案內容 (解決嫌疑三)
ass_path = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original.ass"  # 請確保路徑與你上方的一致
print(f"\n📄 檢查字幕檔內容: {ass_path}")

if os.path.exists(ass_path):
    with open(ass_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        # 印出前 20 行與最後 5 行來檢查
        print("--- [ASS Header & First Events] ---")
        print("".join(lines[:20]))
        print("...")
        print("".join(lines[-5:]))
        print("-----------------------------------")

        # 簡易檢查是否有 Event
        if len(lines) < 15:
            print("⚠️ 警告：字幕檔行數過少，可能沒有生成字幕內容！")
else:
    print("❌ 錯誤：找不到 ASS 檔案！")

# 3. 強制指定字體重新合成 (解決嫌疑一 & 二)
# 我們使用 FFmpeg 的 force_style 來強制覆蓋 ASS 裡的字體設定
print("\n🎬 嘗試使用強制字體設定重新合成...")

video_input = "temp_src.mp4"  # 這是你上一步驟生成的 Wav2Lip 影片
audio_input = (
    "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Godtone_Ver.wav"
)
output_fixed = (
    "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_FIXED.mp4"
)

# 確保暫存檔還在 (如果不在，請重新執行 Wav2Lip 那段，或者直接用上次生成的無字幕影片當輸入)
if not os.path.exists(video_input):
    # 如果暫存檔不見了，嘗試使用最終產出的影片(雖然它沒字幕)當作影像來源
    video_input = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Final_Ultimate.mp4"

if os.path.exists(video_input) and os.path.exists(ass_path):
    # FFmpeg 指令：強制字體為 Noto Sans CJK TC，並調整字體大小以防解析度問題
    # Fontname=Noto Sans CJK TC Regular
    ffmpeg_fix_cmd = [
        "ffmpeg",
        "-y",
        "-v",
        "error",
        "-i",
        video_input,
        "-i",
        audio_input,
        # 關鍵修正：在 ass 過濾器中加入 fontsdir (雖然安裝了，但有時需指定)
        # 或者直接依賴系統字體，這裡我們假設系統字體已安裝成功
        "-vf",
        f"ass='{ass_path}'",
        "-map",
        "0:v",
        "-map",
        "1:a",
        "-c:v",
        "libx264",
        "-preset",
        "medium",
        "-crf",
        "18",
        "-c:a",
        "aac",
        "-b:a",
        "192k",
        "-shortest",
        output_fixed,
    ]

    try:
        subprocess.run(ffmpeg_fix_cmd, check=True)
        print(f"🎉 修復版影片已生成：{output_fixed}")
    except subprocess.CalledProcessError as e:
        print(f"❌ 合成失敗: {e}")
else:
    print("❌ 無法重新合成，找不到來源影片或字幕檔。")

📦 正在安裝中文字體 (Noto Sans CJK)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/opentype: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/opentype/noto: caching, new cache contents: 30 fonts, 0 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 2 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skip

In [7]:
import os
import subprocess

# ================= 1. 設定路徑 =================
ass_path = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original.ass"
video_input = "temp_src.mp4"  # Wav2Lip 的輸出
audio_input = (
    "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Godtone_Ver.wav"
)
final_output = (
    "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_SUCCESS.mp4"
)

# 確保來源存在
if not os.path.exists(video_input):
    # 備用方案：如果暫存檔沒了，用上次那個沒字幕的結果當底
    video_input = "/content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_Final_Ultimate.mp4"

# ================= 2. 手術修復 ASS 檔案 =================
print(f"🔧 正在修復 ASS 檔案格式: {ass_path}")

try:
    with open(ass_path, "r", encoding="utf-8") as f:
        content = f.read()

    # 🔍 搜尋並修復黏在一起的 "TextDialogue"
    if "TextDialogue:" in content:
        print("   ⚠️ 偵測到格式錯誤 (TextDialogue)，正在注入換行符號...")
        # 將 TextDialogue 分割成 Text + 換行 + Dialogue
        fixed_content = content.replace("TextDialogue:", "Text\nDialogue:")

        # 寫回檔案
        with open(ass_path, "w", encoding="utf-8") as f:
            f.write(fixed_content)
        print("   ✅ 檔案已修復！")
    else:
        print("   ℹ️ 檔案看起來正常 (沒有 TextDialogue 黏合問題)，繼續嘗試合成。")

except Exception as e:
    print(f"❌ 讀取檔案失敗: {e}")

# ================= 3. 執行最終合成 (加強版) =================
print("\n🎬 開始最終渲染 (指定字體路徑)...")

# 為了保險起見，我們明確告訴 FFmpeg 字體在哪裡
# Noto Sans CJK 的標準路徑通常在 /usr/share/fonts/opentype/noto
fonts_dir = "/usr/share/fonts/opentype/noto"

ffmpeg_cmd = [
    "ffmpeg",
    "-y",
    "-v",
    "error",
    "-i",
    video_input,
    "-i",
    audio_input,
    # 使用 fontsdir 參數確保 FFmpeg 絕對能找到字體
    "-vf",
    f"ass='{ass_path}':fontsdir='{fonts_dir}'",
    "-map",
    "0:v",
    "-map",
    "1:a",
    "-c:v",
    "libx264",
    "-preset",
    "medium",
    "-crf",
    "18",
    "-c:a",
    "aac",
    "-b:a",
    "192k",
    "-shortest",
    final_output,
]

try:
    subprocess.run(ffmpeg_cmd, check=True)
    print("\n" + "=" * 40)
    print(f"🎉🎉🎉 真・最終完成！請檢查影片：")
    print(f"📂 {final_output}")
    print("=" * 40)
except subprocess.CalledProcessError as e:
    print(f"❌ 合成失敗: {e}")

🔧 正在修復 ASS 檔案格式: /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original.ass
   ⚠️ 偵測到格式錯誤 (TextDialogue)，正在注入換行符號...
   ✅ 檔案已修復！

🎬 開始最終渲染 (指定字體路徑)...

🎉🎉🎉 真・最終完成！請檢查影片：
📂 /content/drive/MyDrive/RVC轉換/Final_Ultimate_Output/demo_original_SUCCESS.mp4
